In [1]:
!pip install underthesea

     |████████████████████████████████| 7.5MB 5.5MB/s 
     |████████████████████████████████| 6.7MB 42.5MB/s 
     |████████████████████████████████| 747kB 35.4MB/s 
     |████████████████████████████████| 245kB 47.3MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [2]:
import regex as re
from underthesea import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pickle
import time
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import numpy as np
MODEL_PATH = "models"

import os
if not os.path.exists(MODEL_PATH):
    os.makedirs(MODEL_PATH)


In [ ]:
# Xử lý văn bản

In [3]:
# Đưa về 1 chuẩn unicode
uniChars = "àáảãạâầấẩẫậăằắẳẵặèéẻẽẹêềếểễệđìíỉĩịòóỏõọôồốổỗộơờớởỡợùúủũụưừứửữựỳýỷỹỵÀÁẢÃẠÂẦẤẨẪẬĂẰẮẲẴẶÈÉẺẼẸÊỀẾỂỄỆĐÌÍỈĨỊÒÓỎÕỌÔỒỐỔỖỘƠỜỚỞỠỢÙÚỦŨỤƯỪỨỬỮỰỲÝỶỸỴÂĂĐÔƠƯ"
unsignChars = "aaaaaaaaaaaaaaaaaeeeeeeeeeeediiiiiooooooooooooooooouuuuuuuuuuuyyyyyAAAAAAAAAAAAAAAAAEEEEEEEEEEEDIIIOOOOOOOOOOOOOOOOOOOUUUUUUUUUUUYYYYYAADOOU"
 
def loaddicchar():
    dic = {}
    char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'.split(
        '|')
    charutf8 = "à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ".split(
        '|')
    for i in range(len(char1252)):
        dic[char1252[i]] = charutf8[i]
    return dic
dicchar = loaddicchar()

# Hàm chuyển Unicode dựng sẵn về Unicde tổ hợp (phổ biến hơn)
def convert_unicode(txt):
    return re.sub(
        r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
        lambda x: dicchar[x.group()], txt)

bang_nguyen_am = [['a', 'à', 'á', 'ả', 'ã', 'ạ', 'a'],
                  ['ă', 'ằ', 'ắ', 'ẳ', 'ẵ', 'ặ', 'aw'],
                  ['â', 'ầ', 'ấ', 'ẩ', 'ẫ', 'ậ', 'aa'],
                  ['e', 'è', 'é', 'ẻ', 'ẽ', 'ẹ', 'e'],
                  ['ê', 'ề', 'ế', 'ể', 'ễ', 'ệ', 'ee'],
                  ['i', 'ì', 'í', 'ỉ', 'ĩ', 'ị', 'i'],
                  ['o', 'ò', 'ó', 'ỏ', 'õ', 'ọ', 'o'],
                  ['ô', 'ồ', 'ố', 'ổ', 'ỗ', 'ộ', 'oo'],
                  ['ơ', 'ờ', 'ớ', 'ở', 'ỡ', 'ợ', 'ow'],
                  ['u', 'ù', 'ú', 'ủ', 'ũ', 'ụ', 'u'],
                  ['ư', 'ừ', 'ứ', 'ử', 'ữ', 'ự', 'uw'],
                  ['y', 'ỳ', 'ý', 'ỷ', 'ỹ', 'ỵ', 'y']]
bang_ky_tu_dau = ['', 'f', 's', 'r', 'x', 'j']

nguyen_am_to_ids = {}

for i in range(len(bang_nguyen_am)):
    for j in range(len(bang_nguyen_am[i]) - 1):
        nguyen_am_to_ids[bang_nguyen_am[i][j]] = (i, j)

# Chuẩn hóa kiểu gõ dấu

def chuan_hoa_dau_tu_tieng_viet(word):
    if not is_valid_vietnam_word(word):
        return word

    chars = list(word)
    dau_cau = 0
    nguyen_am_index = []
    qu_or_gi = False
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x == -1:
            continue
        elif x == 9:  # check qu
            if index != 0 and chars[index - 1] == 'q':
                chars[index] = 'u'
                qu_or_gi = True
        elif x == 5:  # check gi
            if index != 0 and chars[index - 1] == 'g':
                chars[index] = 'i'
                qu_or_gi = True
        if y != 0:
            dau_cau = y
            chars[index] = bang_nguyen_am[x][0]
        if not qu_or_gi or index != 1:
            nguyen_am_index.append(index)
    if len(nguyen_am_index) < 2:
        if qu_or_gi:
            if len(chars) == 2:
                x, y = nguyen_am_to_ids.get(chars[1])
                chars[1] = bang_nguyen_am[x][dau_cau]
            else:
                x, y = nguyen_am_to_ids.get(chars[2], (-1, -1))
                if x != -1:
                    chars[2] = bang_nguyen_am[x][dau_cau]
                else:
                    chars[1] = bang_nguyen_am[5][dau_cau] if chars[1] == 'i' else bang_nguyen_am[9][dau_cau]
            return ''.join(chars)
        return word

    for index in nguyen_am_index:
        x, y = nguyen_am_to_ids[chars[index]]
        if x == 4 or x == 8:  # ê, ơ
            chars[index] = bang_nguyen_am[x][dau_cau]
            return ''.join(chars)

    if len(nguyen_am_index) == 2:
        if nguyen_am_index[-1] == len(chars) - 1:
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
            chars[nguyen_am_index[0]] = bang_nguyen_am[x][dau_cau]
          
        else:
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
            chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
    else:
        x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
        chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
    return ''.join(chars)


def is_valid_vietnam_word(word):
    chars = list(word)
    nguyen_am_index = -1
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x != -1:
            if nguyen_am_index == -1:
                nguyen_am_index = index
            else:
                if index - nguyen_am_index != 1:
                    return False
                nguyen_am_index = index
    return True


def chuan_hoa_dau_cau_tieng_viet(sentence):
    sentence = sentence.lower()
    words = sentence.split()
    for index, word in enumerate(words):
        cw = re.sub(r'(^\p{P}*)([p{L}.]*\p{L}+)(\p{P}*$)', r'\1/\2/\3', word).split('/')
        if len(cw) == 3:
            cw[1] = chuan_hoa_dau_tu_tieng_viet(cw[1])
        words[index] = ''.join(cw)
    return ' '.join(words)

def remove_html(txt):
    return re.sub(r'<[^>]*>', '', txt)

In [21]:
def text_preprocess(document):
    # xóa html code
    document = remove_html(document)
    # chuẩn hóa unicode
    document = convert_unicode(document)
    # chuẩn hóa cách gõ dấu tiếng Việt
    document = chuan_hoa_dau_cau_tieng_viet(document)
    # tách từ
    document = word_tokenize(document, format="text")
    # đưa về lower
    document = document.lower()
    # xóa các ký tự không cần thiết
    document = re.sub(r'[^\s\wáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđ_]',' ',document)
    # xóa khoảng trắng thừa
    document = re.sub(r'\s+', ' ', document).strip()
    return document
document = input()
text_preprocess(document)

Ông Trump viện dẫn cáo buộc gian lận bầu cử để phản đối việc truyền thông Mỹ tính toán và gọi đối thủ Joe Biden là "Tổng thống đắc cử"


'ông trump viện_dẫn cáo_buộc gian_lận bầu_cử để phản_đối việc truyền_thông mỹ tính_toán và gọi đối_thủ joe biden là tổng_thống đắc_cử'

In [5]:
!wget -nc "https://github.com/nguyenvanhieuvn/text-classification-tutorial/raw/master/news_categories.zip"
!unzip -n "news_categories.zip"
!head "news_categories.txt"

--2020-11-15 03:24:46--  https://github.com/nguyenvanhieuvn/text-classification-tutorial/raw/master/news_categories.zip
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/nguyenvanhieuvn/text-classification-tutorial/master/news_categories.zip [following]
--2020-11-15 03:24:46--  https://raw.githubusercontent.com/nguyenvanhieuvn/text-classification-tutorial/master/news_categories.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 59696423 (57M) [application/zip]
Saving to: ‘news_categories.zip’

news_categories.zip 100%[===================>]  56.93M  78.8MB/s    in 0.7s    

2020-11-15 03:24:47 (78.8 MB

In [6]:
file = open("news_categories.txt")

In [7]:
count = {}
for line in file:
    key = line.split()[0]
    count[key] = count.get(key, 0) + 1

for key in count:
    print(key, count[key])

__label__thể_thao 2611
__label__âm_nhạc 2595
__label__nhịp_sống 2613
__label__công_nghệ 2595
__label__thời_sự 2607
__label__thế_giới 2602
__label__thời_trang 2596
__label__du_lịch 2593
__label__sống_trẻ 2602
__label__giáo_dục 2603
__label__kinh_doanh 2597
__label__pháp_luật 2592
__label__giải_trí 2604
__label__phim_ảnh 2596
__label__xe_360 2602
__label__ẩm_thực 2482
__label__xuất_bản 2599
__label__sức_khỏe 2589


In [8]:
total_label = 18
vocab = {}
label_vocab = {}
for line in open('news_categories.txt'):
    words = line.split()
    # lưu ý từ đầu tiên là nhãn
    label = words[0]
    if label not in label_vocab:
        label_vocab[label] = {}
    for word in words[1:]:
        label_vocab[label][word] = label_vocab[label].get(word, 0) + 1
        if word not in vocab:
            vocab[word] = set()
        vocab[word].add(label)

count = {}
for word in vocab:
    if len(vocab[word]) == total_label:
        count[word] = min([label_vocab[x][word] for x in label_vocab])
        
sorted_count = sorted(count, key=count.get, reverse=True)
for word in sorted_count[:100]:
    print(word, count[word])

và 14255
của 13177
là 9983
có 9162
được 9131
trong 8654
một 7575
cho 7483
với 7195
không 6591
các 6300
người 6088
khi 6011
này 5301
đến 5165
để 5123
đã 4431
nhiều 4167
trên 3842
từ 3820
vào 3617
đó 3207
những 3097
ở 2943
ra 2767
tại 2756
vn 2738
lại 2673
cũng 2622
phải 2615
còn 2589
theo 2565
nhưng 2545
zing 2519
về 2170
sau 2047
làm 1983
lên 1921
hơn 1755
đây 1692
năm 1654
sẽ 1597
chỉ 1569
cả 1534
cùng 1490
ngày 1484
như 1467
mà 1458
vẫn 1386
đi 1373
2 1358
mới 1357
khác 1300
3 1296
hai 1278
qua 1250
bạn 1138
bên 1137
1 1136
khiến 1122
5 1114
lần 1051
mình 1045
lớn 1030
bị 1022
biết 1013
trước 1000
rất 991
tới 968
bằng 948
mang 929
nên 897
4 896
đang 870
nước 863
cách 863
việt_nam 861
đầu 849
10 847
việc 840
nếu 835
vừa 826
thấy 824
hàng 807
vì 806
ảnh 799
đều 796
nhau 788
thời_gian 787
sự 764
anh 760
6 737
nhất 733
ngoài 720
điều 712
hay 706
giữa 699
số 699
từng 697
thêm 692


In [9]:
stopword = set()
with open('stopwords.txt', 'w') as fp:
    for word in sorted_count[:100]:
        stopword.add(word)
        fp.write(word + '\n')
    
def remove_stopwords(line):
    words = []
    for word in line.strip().split():
        if word not in stopword:
            words.append(word)
    return ' '.join(words)
    
    
with open('news_categories.prep', 'w') as fp:
    for line in open('news_categories.txt'):
        line = remove_stopwords(line)
        fp.write(line + '\n')

In [10]:
!head news_categories.prep

__label__thể_thao sanchez dự trận khai màn nh arsenal bóng_đá tuyển_thủ chile kịp hồi_phục thể_lực pháo thủ_thành london tham_dự trận đầu_tiên premier_league 2015 2016 diễn 9 8 arsenal singapore liverpool đổ_bộ xuống thái_lan đội_bóng danh_tiếng giải ngoại_hạng_anh bắt_đầu chuyến du_đấu châu á thầy_trò hlv brendan_rodgers có_mặt thái_lan chuẩn_bị trận_đấu diễn 14 7 trả_lời phỏng_vấn chuyến du_đấu singapore hlv arsene_wenger xác_nhận tiền_đạo alexis_sanchez vắng_mặt trận khai màn premier_league chiến_lược gia pháp cựu ngôi_sao barca cần nghỉ_ngơi tham_dự copa_america quê_hương alexis trở_lại arsenal 8 cầu_thủ thường lấy thể_lực khoảng tuần có_thể thi_đấu alexis có_thể nghỉ ít tuy_nhiên ấy chắc_chắn tham_dự trận_đấu đầu_tiên mùa_giải trang bbc dẫn bài phỏng_vấn hlv wenger trận xông đất premier_league sanchez góp_mặt trận tranh community_shield chelsea bởi trận_đấu diễn 8 chân_sút 26 tuổi trở_lại arsenal sanchez nhân_tố then_chốt giúp chile vô_địch copa_america bbc mùa_giải vừa_qua cựu ti

In [11]:
test_percent = 0.2

text = []
label = []

for line in open('news_categories.prep'):
    words = line.strip().split()
    label.append(words[0])
    text.append(' '.join(words[1:]))

X_train, X_test, y_train, y_test = train_test_split(text, label, test_size=test_percent, random_state=42)

# Lưu train/test data
# Giữ nguyên train/test để về sau so sánh các mô hình cho công bằng
with open('train.txt', 'w') as fp:
    for x, y in zip(X_train, y_train):
        fp.write('{} {}\n'.format(y, x))

with open('test.txt', 'w') as fp:
    for x, y in zip(X_test, y_test):
        fp.write('{} {}\n'.format(y, x))

# encode label
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
print(list(label_encoder.classes_), '\n')
y_train = label_encoder.transform(y_train)
y_test = label_encoder.transform(y_test)

print(X_train[0], y_train[0], '\n')
print(X_test[0], y_test[0])

['__label__công_nghệ', '__label__du_lịch', '__label__giáo_dục', '__label__giải_trí', '__label__kinh_doanh', '__label__nhịp_sống', '__label__phim_ảnh', '__label__pháp_luật', '__label__sống_trẻ', '__label__sức_khỏe', '__label__thế_giới', '__label__thể_thao', '__label__thời_sự', '__label__thời_trang', '__label__xe_360', '__label__xuất_bản', '__label__âm_nhạc', '__label__ẩm_thực'] 

long_nhật chê chuyện phương_thanh cứu siu_black sao_việt cái nhìn ngược hoàn_toàn đông long_nhật rằng phương_thanh mọi chuyện rối_tung long_nhật chê chuyện phương_thanh cứu siu_black cái nhìn ngược hoàn_toàn đông long_nhật rằng phương_thanh mọi chuyện rối_tung chuyện siu_black vỡ_nợ thực_sự tạo cơn chấn_động giới giải_trí thay_vì máu_lửa sân_khấu giọng ca núi_rừng xuất_hiện gương_mặt bơ_phờ khóe miệng nở nụ_cười gượng_gạo phương_thanh duy_nhất phát_ngôn báo_chí thừa_nhận khoản nợ tỷ đồng siu_black hoàn_toàn im_lặng thay chị trả_lời tất_cả thắc_mắc giới truyền_thông chính nữ ca_sĩ hâm_mộ gọi cái tên chanh tin ph

In [12]:
start_time = time.time()
text_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1,1),
                                             max_df=0.8,
                                             max_features=None)), 
                     ('tfidf', TfidfTransformer()), 
                     ('clf', MultinomialNB())
                    ])
text_clf = text_clf.fit(X_train, y_train)

train_time = time.time() - start_time
print('Done training Naive Bayes in', train_time, 'seconds.')

# Save model
pickle.dump(text_clf, open(os.path.join(MODEL_PATH, "naive_bayes.pkl"), 'wb'))

Done training Naive Bayes in 12.364723682403564 seconds.


In [13]:
from sklearn.linear_model import LogisticRegression
    
start_time = time.time()
text_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1,1),
                                             max_df=0.8,
                                             max_features=None)), 
                     ('tfidf', TfidfTransformer()),
                     ('clf', LogisticRegression(solver='lbfgs', 
                                                multi_class='auto',
                                                max_iter=10000))
                    ])
text_clf = text_clf.fit(X_train, y_train)

train_time = time.time() - start_time
print('Done training Linear Classifier in', train_time, 'seconds.')

# Save model
pickle.dump(text_clf, open(os.path.join(MODEL_PATH, "linear_classifier.pkl"), 'wb'))

Done training Linear Classifier in 190.93318152427673 seconds.


In [14]:
from sklearn.svm import SVC

start_time = time.time()
text_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1,1),
                                             max_df=0.8,
                                             max_features=None)), 
                     ('tfidf', TfidfTransformer()),
                     ('clf', SVC(gamma='scale'))
                    ])
text_clf = text_clf.fit(X_train, y_train)

train_time = time.time() - start_time
print('Done training SVM in', train_time, 'seconds.')

# Save model
pickle.dump(text_clf, open(os.path.join(MODEL_PATH, "svm.pkl"), 'wb'))

Done training SVM in 2381.5018668174744 seconds.


In [15]:
import numpy as np
# Naive Bayes
model = pickle.load(open(os.path.join(MODEL_PATH,"naive_bayes.pkl"), 'rb'))
y_pred = model.predict(X_test)
print('Naive Bayes, Accuracy =', np.mean(y_pred == y_test))

# Linear Classifier
model = pickle.load(open(os.path.join(MODEL_PATH,"linear_classifier.pkl"), 'rb'))
y_pred = model.predict(X_test)
print('Linear Classifier, Accuracy =', np.mean(y_pred == y_test))

# SVM
model = pickle.load(open(os.path.join(MODEL_PATH,"svm.pkl"), 'rb'))
y_pred = model.predict(X_test)
print('SVM, Accuracy =', np.mean(y_pred == y_test))


Naive Bayes, Accuracy = 0.8324764353041988
Linear Classifier, Accuracy = 0.8832476435304198
SVM, Accuracy = 0.8822836332476436


In [16]:
from sklearn.metrics import classification_report

nb_model = pickle.load(open(os.path.join(MODEL_PATH,"naive_bayes.pkl"), 'rb'))
y_pred = nb_model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=list(label_encoder.classes_)))

                     precision    recall  f1-score   support

 __label__công_nghệ       0.91      0.92      0.92       532
   __label__du_lịch       0.79      0.88      0.83       551
  __label__giáo_dục       0.82      0.88      0.85       528
  __label__giải_trí       0.59      0.74      0.66       487
__label__kinh_doanh       0.78      0.84      0.81       498
 __label__nhịp_sống       0.85      0.49      0.62       497
  __label__phim_ảnh       0.90      0.76      0.82       525
 __label__pháp_luật       0.90      0.92      0.91       543
  __label__sống_trẻ       0.62      0.64      0.63       510
  __label__sức_khỏe       0.79      0.88      0.83       496
  __label__thế_giới       0.91      0.83      0.87       549
  __label__thể_thao       0.95      0.95      0.95       508
   __label__thời_sự       0.82      0.77      0.79       496
__label__thời_trang       0.86      0.77      0.81       521
    __label__xe_360       0.97      0.94      0.96       502
  __label__xuất_bản    

In [17]:
from sklearn.metrics import classification_report
nb_model1 = pickle.load(open(os.path.join(MODEL_PATH,"linear_classifier.pkl"), 'rb'))
y_pred = nb_model1.predict(X_test)
print(classification_report(y_test, y_pred, target_names=list(label_encoder.classes_)))

                     precision    recall  f1-score   support

 __label__công_nghệ       0.94      0.94      0.94       532
   __label__du_lịch       0.88      0.90      0.89       551
  __label__giáo_dục       0.86      0.88      0.87       528
  __label__giải_trí       0.76      0.77      0.76       487
__label__kinh_doanh       0.86      0.86      0.86       498
 __label__nhịp_sống       0.81      0.79      0.80       497
  __label__phim_ảnh       0.87      0.87      0.87       525
 __label__pháp_luật       0.94      0.92      0.93       543
  __label__sống_trẻ       0.77      0.72      0.74       510
  __label__sức_khỏe       0.86      0.90      0.88       496
  __label__thế_giới       0.90      0.91      0.90       549
  __label__thể_thao       0.96      0.97      0.96       508
   __label__thời_sự       0.85      0.85      0.85       496
__label__thời_trang       0.88      0.88      0.88       521
    __label__xe_360       0.95      0.97      0.96       502
  __label__xuất_bản    

In [18]:
nb_model2 = pickle.load(open(os.path.join(MODEL_PATH,"linear_classifier.pkl"), 'rb'))
y_pred = nb_model2.predict(X_test)
print(classification_report(y_test, y_pred, target_names=list(label_encoder.classes_)))

                     precision    recall  f1-score   support

 __label__công_nghệ       0.94      0.94      0.94       532
   __label__du_lịch       0.88      0.90      0.89       551
  __label__giáo_dục       0.86      0.88      0.87       528
  __label__giải_trí       0.76      0.77      0.76       487
__label__kinh_doanh       0.86      0.86      0.86       498
 __label__nhịp_sống       0.81      0.79      0.80       497
  __label__phim_ảnh       0.87      0.87      0.87       525
 __label__pháp_luật       0.94      0.92      0.93       543
  __label__sống_trẻ       0.77      0.72      0.74       510
  __label__sức_khỏe       0.86      0.90      0.88       496
  __label__thế_giới       0.90      0.91      0.90       549
  __label__thể_thao       0.96      0.97      0.96       508
   __label__thời_sự       0.85      0.85      0.85       496
__label__thời_trang       0.88      0.88      0.88       521
    __label__xe_360       0.95      0.97      0.96       502
  __label__xuất_bản    

In [19]:
document = input()

document = text_preprocess(document)
document = remove_stopwords(document)

label = nb_model.predict([document])
label1 = nb_model1.predict([document])
label2 = nb_model2.predict([document])
print('Predict label:', label_encoder.inverse_transform(label))
print("--------------------------------")
print('Predict label:', label_encoder.inverse_transform(label1))
print("--------------------------------")
print('Predict label:', label_encoder.inverse_transform(label2))

ông joe biden trở thành tổng thống thứ 46 của mỹ
Predict label: ['__label__thế_giới']
--------------------------------
Predict label: ['__label__thế_giới']
--------------------------------
Predict label: ['__label__thế_giới']
